In [15]:
%load_ext autoreload
%autoreload 2

import os
os.environ['TORCH_HOME'] = "/disks/sdb/torch_home"

import logging
logging.getLogger().setLevel(logging.WARNING)

from awesome_glue.task import *
from awesome_glue.config import Config

Loading faiss with AVX2 support.


In [17]:
config = Config()
task = Task(config)
task.from_pretrained()
config

cache for AGNEWS-spacy.data exists
*** load AGNEWS-spacy.data from cache cost 21.5 seconds
cache for AGNEWS-glove.vec exists
*** load AGNEWS-glove.vec from cache cost 0.0296 seconds
Load model from saved/models/AGNEWS-lstm/best.th


Basic Args:
	--task_id=AGNEWS
	--finetunable=True
	--arch=lstm
	--pretrain=glove
	--_model_name=
	--mode=attack
	--aug_data=
	--adv_iter=0
	--adv_policy=rdm
	--adv_replace_num=20
	--adv_constraint=True
	--pred_ensemble=1
	--pred_transform=
	--pred_transform_args=
	--attack_method=pwws
	--attack_vectors=counter
	--attack_data_split=dev
	--attack_size=200
	--attack_gen_adv=False
	--adv_data=nogit/AGNEWS-lstm.hotflip.adv.tsv
	--alchemist=False
	--seed=2
	--cuda=0
Deduced Args:
	--model_name=AGNEWS-lstm
	--tokenizer=spacy

In [30]:
spacy_vocab, spacy_vec = task.get_spacy_vocab_and_vec()
searcher = EmbeddingSearcher(spacy_vec, 
                            word2idx=spacy_vocab.get_token_index,
                            idx2word=spacy_vocab.get_token_from_index)

cache for AGNEWS-counter.vec exists
*** load AGNEWS-counter.vec from cache cost 0.0318 seconds


In [32]:
searcher.search('index', topk=10, return_words=True)

['index',
 'indexed',
 'clue',
 'hint',
 'indicator',
 'indication',
 'indicators',
 'pegged',
 'pointers',
 'signposts']

In [7]:
from awesome_glue.transforms import parse_transform_fn_from_args


In [14]:
fn = parse_transform_fn_from_args('rand_drop|embed_aug', '0.3|0.9')
fn(['Stock Prices Climb Ahead of Google IPO NEW YORK - Investors shrugged off rising crude futures Wednesday to capture well'])

['Prices Climb Ahead di Google IPO NEW YORK - muttered contracts Wednesday aux bene']

In [38]:

task.predictor.predict("Stock Prices Climb Ahead of Google IPO NEW YORK - Investors shrugged off rising crude futures Wednesday to capture well - priced shares , sending the Nasdaq composite index up 1.6 percent ahead of Google Inc. 's much - anticipated initial public offering of stock . In afternoon trading , the Dow Jones industrial average gained 67.10 , or 0.7 percent , to 10,039.93 ...")

{'probs': [0.6894714832305908,
  0.0003360987757332623,
  0.30292391777038574,
  0.007268466055393219]}

In [87]:
config = Config()
config.task_id = 'AGNEWS'
task = Task(config)
task.from_pretrained()
config

values = list(task.vocab.get_index_to_token_vocabulary().values())
results = []
from allennlp.common.util import lazy_groups_of
for group in lazy_groups_of(values, 1024):
    result = task.predictor.predict_batch_json([{"sent": ele} for ele in group])
    result = [ele['probs'][0] for ele in result]
    results.extend(result)
pairs = sorted(zip(values, results), key=lambda x: x[1], reverse=False)

cache for AGNEWS-spacy.data exists
*** load AGNEWS-spacy.data from cache cost 18.8 seconds
cache for AGNEWS-glove.vec exists
*** load AGNEWS-glove.vec from cache cost 0.0567 seconds


In [90]:
config._model_name = 'AGNEWS-lstm-hot.1.5.con'
task = Task(config)
task.from_pretrained()
task.model.eval()
task.evaluate_model()
config

values = list(task.vocab.get_index_to_token_vocabulary().values())
results = []
from allennlp.common.util import lazy_groups_of
for group in lazy_groups_of(values, 1024):
    result = task.predictor.predict_batch_json([{"sent": ele} for ele in group])
    result = [ele['probs'][0] for ele in result]
    results.extend(result)
pairs2 = sorted(zip(values, results), key=lambda x: x[1], reverse=False)

cache for AGNEWS-spacy.data exists
*** load AGNEWS-spacy.data from cache cost 17.6 seconds
cache for AGNEWS-glove.vec exists
*** load AGNEWS-glove.vec from cache cost 0.042 seconds



{'accuracy': 0.8999332888592395, 'loss': 0.3075038285014477}


In [91]:
for ele in pairs[:30]:
    for ele2 in pairs2:
        if ele[0] == ele2[0]:
            print(ele[0], round(ele[1], 2), round(ele2[1], 2))

tickers 0.0 0.01
Forbes.com 0.0 0.0
peripherals 0.0 0.04
camcorder 0.0 0.09
fax 0.0 0.07
lender 0.0 0.07
KLM 0.0 0.01
WMT.N 0.0 0.01
lifecycle 0.0 0.01
G20 0.0 0.01
securities 0.0 0.08
interface 0.0 0.02
reinsurance 0.0 0.03
quickinfo 0.0 0.0
investing 0.0 0.05
wireless 0.0 0.03
router 0.0 0.03
UAIR.O 0.0 0.01
retirees 0.0 0.07
generates 0.0 0.19
www.investor.reuters.com 0.0 0.01
55-a 0.0 0.01
mortgage 0.0 0.05
inflationary 0.0 0.03
companys 0.0 0.02
ConocoPhillips 0.0 0.01
subscriptions 0.0 0.04
bookkeeping 0.0 0.06
spinner 0.0 0.08
subsystem 0.0 0.01


In [92]:
print(list(map(lambda x: x[0], pairs[:30])))
print(list(map(lambda x: x[0], pairs2[:30])))

['tickers', 'Forbes.com', 'peripherals', 'camcorder', 'fax', 'lender', 'KLM', 'WMT.N', 'lifecycle', 'G20', 'securities', 'interface', 'reinsurance', 'quickinfo', 'investing', 'wireless', 'router', 'UAIR.O', 'retirees', 'generates', 'www.investor.reuters.com', '55-a', 'mortgage', 'inflationary', 'companys', 'ConocoPhillips', 'subscriptions', 'bookkeeping', 'spinner', 'subsystem']
['HealthDay', 'swappers?', 'freeware', '700th', 'NEERGAARD', 'humphead', '8482', 'iPod', 'EMMA', 'LAURAN', "Cash'n'Carrion", 'RFID', 'Conservation', 'Unix', 'handhelds', 'Playstation', 'shareware', 'fourball', 'Gamers', 'Basketball', 'NHL', 'configurator', 'X-43A', 'Forrester', 'Jaschan', 'Shaquille', 'Email', 'Kobe', 'Vitali', 'aggregator']


In [30]:
toy_data = load_data("TOY", "spacy")
vocab = toy_data['vocab']

cache for TOY-spacy.data exists
*** load TOY-spacy.data from cache cost 4.25 seconds


In [3]:
from allennlp.data.tokenizers import PretrainedTransformerTokenizer
tokenizer = PretrainedTransformerTokenizer('bert-base-uncased')
tokenizer.tokenize('hello world i want to fly onto the dell')

I0320 08:16:04.904309 140473276524352 configuration_utils.py:254] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /disks/sdb/torch_home/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0320 08:16:04.906084 140473276524352 configuration_utils.py:290] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "eos_token_ids": 0,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 5

[[CLS], hello, world, i, want, to, fly, onto, the, dell, [SEP]]